In [1]:
import platform
print(platform.python_version())

3.7.1


In [2]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import to_timestamp, unix_timestamp, max as max_dttm
from pyspark.sql.types import *
spark = SparkSession.builder.appName('PySpark').getOrCreate()
print(spark.version)

2.4.0


In [44]:
df = spark.read.csv('chicago_crimes_2001_to_present.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [45]:
from pyspark.sql.functions import year, month, dayofweek, dayofyear, dayofmonth, weekofyear
df = df.withColumn('dttm', unix_timestamp('Date', "MM/dd/yyyy hh:mm:ss aa").cast(TimestampType()))
# df = df.withColumn('dayofweek', dayofweek('dttm')).withColumn('dayofyear', dayofyear('dttm'))
# df = df.withColumn('year', year('dttm'))
# df = df.withColumn('weekofyear', weekofyear('dttm'))
# df = df.withColumn('month', month('dttm'))
# df = df.withColumn('dayofmonth', dayofmonth('dttm'))
df.show()


+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------------------+---------+---------+----------+-----+----------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|year|          Updated On|    Latitude|    Longitude|            Location|               dttm|dayofweek|dayofyear|weekofyear|month|dayofmonth|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+----------

In [46]:
total_entries = df.count()
print('Total Entries: ' + str(total_entries))

Total Entries: 6754279


In [9]:
first_date = df.agg({"dttm":"min"}).collect()[0][0]
print('First Date: ' + str(first_date))

last_date = df.agg({"dttm":"max"}).collect()[0][0]
print('Last Date: ' + str(last_date))

First Date: 2001-01-01 00:00:00
Last Date: 2018-11-22 23:56:00
Total Entries: 6754279


In [35]:
from pyspark.sql.functions import year, month, dayofweek, dayofyear, dayofmonth
# total_years = df.select(year('dttm').alias('distinct_years')).groupBy('distinct_years').count()
# total_years.show()

from pyspark.sql.functions import col, countDistinct
# df.agg(countDistinct(col(year("dttm"))).alias("count")).show()

gr = df.groupBy(year('dttm')).agg(countDistinct(year('dttm'))).na.drop()
gr.show()
print(gr.count())
print('xx')
gr = df.groupBy(year('dttm')).count().show()
print(gr)
print('yy')
gr = df.agg(countDistinct(year('dttm')))
gr.show()
print(gr.collect()[0][0])

print('mm')
gr = df.agg(countDistinct(month('dttm')))
gr.show()
print(gr.collect()[0][0])


print('dd')
gr = df.agg(countDistinct(dayofmonth('dttm')))
gr.show()
print(gr.collect()[0][0])

# gr = df.agg(countDistinct(month('dttm'))).show()
# print(gr.collect()[0][0])

# gr = df.agg(countDistinct(dayofmonth('dttm'))).show()
# print(gr.collect()[0][0])


+----------+--------------------------+
|year(dttm)|count(DISTINCT year(dttm))|
+----------+--------------------------+
|      2003|                         1|
|      2007|                         1|
|      2018|                         1|
|      2015|                         1|
|      2006|                         1|
|      2013|                         1|
|      2014|                         1|
|      2004|                         1|
|      2012|                         1|
|      2009|                         1|
|      2016|                         1|
|      2001|                         1|
|      2005|                         1|
|      2010|                         1|
|      2011|                         1|
|      2008|                         1|
|      2017|                         1|
|      2002|                         1|
+----------+--------------------------+

18
xx
+----------+------+
|year(dttm)| count|
+----------+------+
|      2003|475923|
|      2007|436986|
|      2018|2

In [22]:
from pyspark.sql.functions import year
year_general = df.select(year('dttm').alias('year')).groupBy('year').count().orderBy('year')
year_general.show()
# year_general_max = year_general.agg({"year":"max"}).collect()[0][0]
# print(year_general_max)
# year_general.filter(year_general['count'] == year_general_max).show()
# print(year_general.agg({"year":"min"}).collect()[0][0])
# print(year_general.agg({"year":"avg"}).collect()[0][0])

+----+------+
|year| count|
+----+------+
|null|   247|
|2001|485738|
|2002|486742|
|2003|475923|
|2004|469366|
|2005|453686|
|2006|448078|
|2007|436986|
|2008|427024|
|2009|392649|
|2010|370277|
|2011|351736|
|2012|335908|
|2013|307020|
|2014|275196|
|2015|263930|
|2016|268814|
|2017|267811|
|2018|237148|
+----+------+



In [7]:
max_value = year_general.agg({"count":"max"}).collect()[0][0]
max_year = year_general.filter(year_general['count'] == max_value).collect()[0][0]
print(str(max_year) + ": " + str(max_value))

2002: 486742
